# Configurable Transformer Video - Example

## Helper methods for Feature Model generation

In [1]:
# Add variables to the FM
def add_fm_variables(model):
    variables[model["id"]] = Var(True, False, id=model["id"])
    for child in model["children"]:
        add_fm_variables(child)

# Add constraints to the FM
def add_fm_constraints(model, parent):
    if model["type"] == "root":
        satisfy(variables[model["id"]] == 1)
    elif model["type"] == "mandatory" and model["relation"] == "":
        satisfy(variables[model["id"]] == variables[model["parent"]])
    elif model["type"] == "optional" and model["relation"] == "":
        satisfy(imply(variables[model["id"]], variables[model["parent"]]))
    elif (model["type"] == "mandatory" or model["type"] == "optional") and (model["relation"] == "alternative" or model["relation"] == "or"):
        all_children_ids = [c["id"] for c in parent["children"]]
        all_children_vars = [~variables[cid] for cid in all_children_ids]

        if model["relation"] == "or":
            satisfy(variables[parent["id"]] == disjunction(*all_children_vars))
        elif model["relation"] == "alternative":
            all_children_vars_orders = []
            i = 0
            while i < len(all_children_vars):
                if i == 0:
                    all_children_vars_orders.append(~all_children_vars[i] == conjunction([*all_children_vars[i + 1 : ], variables[parent["id"]]]))
                elif i == len(all_children_vars) - 1:
                    all_children_vars_orders.append(~all_children_vars[i] == conjunction([*all_children_vars[0 : -1], variables[parent["id"]]]))
                else:
                    all_children_vars_orders.append(~all_children_vars[i] == conjunction([*all_children_vars[0 : i], *all_children_vars[i + 1 : ], variables[parent["id"]]]))
                i += 1
            satisfy(disjunction(*all_children_vars_orders))

    for r in model["requires"]:
        satisfy(imply(variables[model["id"]], variables[r]))

    for e in model["excludes"]:
        satisfy(~variables[model["id"]] | variables[e])

    for child in model["children"]:
        add_fm_constraints(child, model)

# Add variables and constraints for the video duration to the FM
def add_fm_duration_variables_and_constraints(videos_data):
    duration_values = [v["duration"] for v in videos_data.values()]
    duration_keys = []

    for video_key in videos_data.keys():
        name = f"{video_key}_duration"
        duration_keys.append(name)
        variables[name] = Var(0, videos_data[video_key]["duration"], id=name)
        satisfy((variables[video_key] == 1) == (variables[name] > 0))

    variables["max_duration"] = Var(range(0, sum(duration_values)), id="max_duration")
    satisfy(sum([variables[d] for d in duration_keys]) <= variables["max_duration"])

# Adds the user preference for duration
def add_duration_preference(max_duration):
    satisfy(variables["max_duration"] == max_duration)

## Load video files and configuration model

In [2]:
import json

# Video file
with open("TransformerExample/videos.json", "r") as videos_file:
    videos = json.load(videos_file)

# Configuration model
with open("TransformerExample/model.json", "r") as model_file:
    video_model = json.load(model_file)

## Initialize the FM

In [3]:
from pycsp3 import *

# Reset FM
clear()

# Define the FM
variables = {}
add_fm_variables(video_model)
add_fm_duration_variables_and_constraints(videos)
add_fm_constraints(video_model, None)

# Define the user preference for duration
duration_preference = 500
add_duration_preference(duration_preference)

## Find solutions

In [ ]:
if solve(sols=ALL) is SAT:
    print(f"Total number of solutions {n_solutions()}")
    solutions = []
    for i in range(n_solutions()):
        video_keys = []
        playlist = []
        for key in videos.keys():
            if value(variables[key], sol=i):
                video_keys.append(key)
                playlist.append(videos[key]["url"])
        solutions.append({
            "total_duration": sum([videos[k]["duration"] for k in video_keys]),
            "video_keys": video_keys,
            "playlist": playlist
        })

## Inspect solutions
All solutions are persisted in 'solutions_df'. To determine which of those should be returned, different heuristics are applicable.

In [5]:
import pandas as pd
solutions_df = pd.DataFrame(solutions)

### Return shortest version

In [ ]:
shortest_configuration = solutions_df.sort_values('total_duration', ascending=True).iloc[0]
print(f"Video Keys: {shortest_configuration['video_keys']} \nPlaylist: {shortest_configuration['playlist']} \nTotal Duration: {shortest_configuration['total_duration']} s")

### Return longest version


In [ ]:
longest_configuration = solutions_df.sort_values('total_duration', ascending=False).iloc[0]
print(f"Video Keys: {longest_configuration['video_keys']} \nPlaylist: {longest_configuration['playlist']} \nTotal Duration: {longest_configuration['total_duration']} s")